# Objective: 
Given a protein family dataframe (obtained from UniProt), fetch Rhea identifiers. 

In [1]:
import pandas as pd
import os
import re
import seaborn as sns
import numpy as np

In [2]:
# file_in = '../../data/mohammed/rhea_test/uniprot-ipr002347.tab'
# file_in = '../../data/mohammed/uniprot-ipr002347_annot345.csv'
file_in = '../../data/mohammed/uniprot-ipr029063_reviewed.tab'
df = pd.read_csv(file_in, sep = '\t')

In [3]:
df.columns

Index(['Entry', 'Entry name', 'Status', 'Protein names', 'Gene names',
       'Organism', 'Length', 'Annotation', 'Rhea ID', 'EC number',
       'Catalytic activity', 'ChEBI', 'Sequence'],
      dtype='object')

In [4]:
df_45 = df[(df.Annotation.values == '4 out of 5') | (df.Annotation.values == '5 out of 5')].copy()
print(df_45.shape)

df_345 = df[(df.Annotation.values == '4 out of 5') | (df.Annotation.values == '5 out of 5') | (df.Annotation.values == '3 out of 5')].copy()
print(df_345.shape)

(1252, 13)
(5348, 13)


In [5]:
l_cutoff = 500

df_45_l = df_45[df_45.Length.values <= l_cutoff].copy()
print(df_45_l.shape)

df_345_l = df_345[df_345.Length.values <= l_cutoff].copy()
print(df_345_l.shape)

(832, 13)
(4590, 13)


In [6]:
print(df_45_l[df_45_l.Organism.str.contains('tuberculosis')].shape)
print(df_345_l[df_345_l.Organism.str.contains('tuberculosis')].shape)

(16, 13)
(80, 13)


### Fetching Rhea IDs: 

In [7]:
df_45_l.shape

(832, 13)

In [8]:
df_45_l.fillna({'Rhea ID': 'no_ID'}, inplace=True)

In [9]:
df_45_l[df_45_l['Rhea ID'].values == 'no_ID'].shape

(82, 13)

In [10]:
list_up_entry = []
list_rhea_ids = []
for index, row in df_45_l.iterrows():
    up_entry = row['Entry']
    if row['Rhea ID'] == 'no_ID':
        rhea_ids = ['no_ID']
    else:
        rhea_ids = row['Rhea ID'].split(';')
        rhea_ids = [rid.split(':')[1] for rid in rhea_ids]
    list_rhea_ids+=rhea_ids
    list_up_entry+=len(rhea_ids)*[up_entry]

df_45_l_rhea = pd.DataFrame()
df_45_l_rhea['Entry'] = list_up_entry
df_45_l_rhea['RheaID'] = list_rhea_ids

In [11]:
file_out = '../../data/mohammed/rhea_test/up_to_rhea_SAM_45.csv'
df_45_l_rhea.to_csv(file_out, index = False)

In [12]:
df_45_l_rhea.head()

,Entry,RheaID
0,P21964,17877
1,P21964,17878
2,P21964,53108
3,P21964,53109
4,P21964,53104


In [14]:
df_45_l_rhea.RheaID.value_counts().head(30)

28286    250
42640    246
no_ID     82
48096     23
17877     16
12721     16
42708     13
24604     12
24605     12
54712     11
54736     11
54724     11
67008     10
12705     10
15573     10
20225     10
60560      9
60561      9
50904      9
36899      9
43152      9
44380      8
16925      7
11988      7
20945      7
44452      7
20944      7
10281      7
10280      7
53108      6
Name: RheaID, dtype: int64